In [ ]:
# !pip install catboost optuna

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.feature_selection import  SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC,NuSVC
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
import catboost as ctb
import lightgbm as gbm
import optuna
import warnings
warnings.filterwarnings('ignore')

# import sys
# sys.path.append('../')

# # import basic_preprocessing.ipynb
# from preprocessing.preprocessing import   *
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, plot_confusion_matrix
from sklearn.metrics import cohen_kappa_score, accuracy_score, roc_auc_score, f1_score


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN

from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from scipy.stats import boxcox, mode, zscore

import os
import sys


def log_transform(df, cols):
    for col in cols:
        df[col] = np.log1p(df[col])
    return df


def sqrt_transform(df, cols):
    for col in cols:
        df[col] = np.sqrt(df[col])
    return df


def boxcox_transform(df, cols):
    for col in cols:
        df[col], _ = boxcox(df[col])
    return df


def feature_log_transform(df, cols):
    for col in cols:
        df[col+"_log"] = np.log1p(df[col])
    return df


def feature_sqrt_transform(df, cols):
    for col in cols:
        df[col+'_sqrt'] = np.sqrt(df[col])
    return df


def feature_boxcox_transform(df, cols):
    for col in cols:
        df[col+"_boxcox"], _ = boxcox(df[col])
    return df


def remove_outliers(df, cols):
    for col in cols:
        q_low = df[col].quantile(0.01)
        q_hi = df[col].quantile(0.99)
        df_filtered = df[(df[col] < q_hi) & (df[col] > q_low)]

    return df_filtered


def remove_outliers_zscore(df):
    z_scores = zscore(df)
    # calculate z-scores of `df`

    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    new_df = df[filtered_entries]

    return new_df


def features_target(df, features):
    X = df[features]
    y = df['y']

    return X, y


def float_of_column(df, cols):
    for col in cols:
        colname = col+"_float"
        intcol = df[col].astype(int)
        df[colname] = df[col] - intcol
    return df

# # use smote to upsample impalanced class


def upsample_minority(df, features):

    df_X = df[features]
    df_y = df['y']
    sm = SMOTE()

    df_X, df_y = sm.fit_resample(df_X, df_y)
    df = pd.merge(df_X, df_y, left_index=True, right_index=True)

    return df


def upsample_minority_BorderlineSMOTE(df, features):

    df_X = df[features]
    df_y = df['y']
    sm = BorderlineSMOTE()

    df_X, df_y = sm.fit_resample(df_X, df_y)
    df = pd.merge(df_X, df_y, left_index=True, right_index=True)

    return df


def upsample_minority_ADASYN(df, features):

    df_X = df[features]
    df_y = df['y']
    sm = ADASYN()

    df_X, df_y = sm.fit_resample(df_X, df_y)
    df = pd.merge(df_X, df_y, left_index=True, right_index=True)

    return df

# replace avg with mode


def add_cluster_feature(df, n):
    labels = np.zeros((df.shape[0], n))

    for i in range(n):
        km = KMeans(n_clusters=7, random_state=i)
        km.fit(df)
        labels[:, i] = km.predict(df)

    final_labels = mode(labels, axis=1)[0]
    return final_labels


def add_feature_area_minus_convex(df):
    df['remaining_area'] = np.abs(df['ConvexArea'] - df['Area'])
    return df


def add_feature_shapeFactor5(df):
    df['shapeFactor5'] = df['Area'] / \
        ((((df['MinorAxisLength'])**2)/4) * np.pi)
    return df


def add_feature_curl(df):
    df['fibre_length'] = (
        df['Perimeter'] - np.sqrt(np.abs(df['Perimeter']**2 - (16*df['Area'])))) / 4

    df['fibre_width'] = df['Area'] / df['fibre_length']
    df['curl'] = df['MajorAxisLength'] / df['fibre_length']

    return df




In [ ]:
if 'kaggle' in os.environ.get('KAGGLE_URL_BASE','localhost'):
    print("kaggle")
    dataset_path = '../input/dry-beans-classification-iti-ai-pro-intake01/'
    train = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
    test = pd.read_csv(os.path.join(dataset_path, 'test.csv'))

else:
    print("localhost")
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')

In [ ]:
features = train.columns.delete([0,-1])
X,y = features_target(train,features)

In [ ]:
float_cols = ['Perimeter','MajorAxisLength','MinorAxisLength','EquivDiameter']
# skewed_cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
#        'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent',
#        'Solidity', 'roundness', 'ShapeFactor2',
#        'ShapeFactor4']
skewed_cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
       # 'AspectRation', 'Eccentricity',
        'ConvexArea', 
       # 'EquivDiameter', 'Extent',
       'Solidity', 'roundness', 'ShapeFactor2',
       'ShapeFactor4']


In [ ]:
ss = StandardScaler()
mm= MinMaxScaler()
# scaled_train = ss.fit_transform(train[features])
# scaled_test = ss.transform(test[features])
ids = test['ID']
y = train.y
train = train.drop(['ID','y'], axis=1)
test = test.drop(['ID'], axis=1)

ss_train = ss.fit_transform(train)
ss_train_df = pd.DataFrame(ss_train,columns=train.columns)
ss_test = ss.transform(test)
ss_test_df = pd.DataFrame(ss_test,columns=test.columns)

mm_train = mm.fit_transform(train)
mm_train_df = pd.DataFrame(ss_train,columns=train.columns)
mm_test = mm.transform(test)
mm_test_df = pd.DataFrame(ss_test,columns=test.columns)


In [ ]:

def pipline_train(df):
    # df = float_of_column(df,float_cols)

    # df = remove_outliers(df,list(features))
#     df = upsample_minority(df,features)

    # # X,y = features_target(df,features)
    labels  = add_cluster_feature(mm_train_df,1)
    df['labels'] = labels
    # # df = add_feature_curl(df)
    # df = add_feature_area_minus_convex(df)
    # df = add_feature_shapeFactor5(df)
    # X,y = features_target(df,features)
    # df = remove_outliers_zscore(X)
    # df['y'] = y


    # # df = log_transform(df,skewed_cols)
    # # df = sqrt_transform(df,skewed_cols)
    # df = boxcox_transform(df,skewed_cols)

    # df = feature_boxcox_transform(df,skewed_cols)
    # df = feature_log_transform(df,skewed_cols)
    # df = feature_sqrt_transform(df,skewed_cols)

    
    return df
def pipline_test(df):
    # df = float_of_column(df,float_cols)
    labels  = add_cluster_feature(mm_test_df,1)
    df['labels'] = labels
    # # df = add_feature_curl(df)
    # df = add_feature_area_minus_convex(df)
    # df = add_feature_shapeFactor5(df)

    # # df = log_transform(df,skewed_cols)
    # # df = sqrt_transform(df,skewed_cols)
    # df = boxcox_transform(df,skewed_cols)
    # df = feature_boxcox_transform(df,skewed_cols)
    # df = feature_log_transform(df,skewed_cols)
    # df = feature_sqrt_transform(df,skewed_cols)    


    return df



train = pipline_train(train);
test  = pipline_test(test);



In [ ]:
pd.crosstab(ss_train_df['lablels'],y)

In [ ]:
ss_train_df['lablels'] =train['labels']
ss_test_df['lablels'] = test['labels']

mm_train_df['lablels'] = train['labels']
mm_test_df['lablels'] =  test['labels']

In [ ]:
mm_train_df

In [ ]:
features = mm_train_df.columns
X,y = mm_train_df,y

In [ ]:
sira_dermason = mm_train_df[(y == 'SIRA') | (y == 'DERMASON') ]
# sira_dermason_model = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
#               beta_2=0.999, early_stopping=False, epsilon=1e-08,
#               hidden_layer_sizes=(100,50,), learning_rate='adaptive',
#               learning_rate_init=0.001, max_fun=15000, max_iter=500,
#               momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
#               power_t=0.5, random_state=42, shuffle=True, solver='adam',
#               tol=0.0001, validation_fraction=0.2, verbose=False,
#               warm_start=False)
# sira_dermason_model = SVC()
sira_dermason_model = ctb.CatBoostClassifier(random_state = 42, verbose=0,thread_count=-1)
# sira_dermason_model = KNeighborsClassifier(n_neighbors=20)
# sira_dermason_model = xgb.XGBClassifier(random_state = 42, verbosity=0)
X_sira_dermason,y_sira_dermason = sira_dermason, y[(y == 'SIRA') | (y == 'DERMASON') ]
# xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,random_state=42,test_size=0.2)
# sira_dermason_model.fit(xtrain,ytrain)
# preds = sira_dermason_model.predict(xtrain)
# preds2 = sira_dermason_model.predict(xvalid)
# print("accuracy is {}".format(accuracy_score(ytrain,preds)))
# print("accuracy is {}".format(accuracy_score(yvalid,preds2)))



kf = KFold(n_splits=5,random_state=1,shuffle=True)
i=1
# scores=[]
for train_index, test_index in kf.split(X_sira_dermason,y_sira_dermason):
#         print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_sira_dermason.iloc[train_index], X_sira_dermason.iloc[test_index]
        y_train, y_test = y_sira_dermason.iloc[train_index], y_sira_dermason.iloc[test_index]
        # for model in models:
        sira_dermason_model.fit(X_train,y_train)
        
        preds_train = sira_dermason_model.predict(X_train)        
        preds_test = sira_dermason_model.predict(X_test)
        print("fold {}  train accuracy is {}".format(i,accuracy_score(y_train,preds_train)))
                        
        print("fold {}  test accuracy is {}".format(i,accuracy_score(y_test,preds_test)))

        # scores.append(accuracy_score(y_test,preds_test))
        i+=1
# fold 1  train accuracy is 0.9236602628918099
# fold 1  test accuracy is 0.9171717171717172
# fold 2  train accuracy is 0.9257012888551933
# fold 2  test accuracy is 0.9191102123356926
# fold 3  train accuracy is 0.931008339651251
# fold 3  test accuracy is 0.9049544994944388
# fold 4  train accuracy is 0.9226686884003032
# fold 4  test accuracy is 0.9251769464105156
# fold 5  train accuracy is 0.9206469547637098
# fold 5  test accuracy is 0.9322548028311426

sira_dermason_model.fit(X_sira_dermason,y_sira_dermason)

In [ ]:
# model = ctb.CatBoostClassifier(random_state = 42, verbose=0,thread_count=-1)
# model = K
model = xgb.XGBClassifier(random_state = 42, verbosity=0,n_jobs=-1)
# model = SVC()
X,y = mm_train_df,y
# model = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
#               beta_2=0.999, early_stopping=False, epsilon=1e-08,
#               hidden_layer_sizes=(200,), learning_rate='adaptive',
#               learning_rate_init=0.001, max_fun=15000, max_iter=500,
#               momentum=0.8, n_iter_no_change=10, nesterovs_momentum=True,
#               power_t=0.5, random_state=42, shuffle=True, solver='adam',
#               tol=0.0001, validation_fraction=0.2, verbose=False,
#               warm_start=False)
# X,y = features_target(train,features)
kf = KFold(n_splits=5,random_state=1,shuffle=True)
i=1
# scores=[]
for train_index, test_index in kf.split(X,y):
#         print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        # for model in models:
        
        model.fit(X_train,y_train)
        preds_test = model.predict(X_test)

#         cm = confusion_matrix(y_test,preds_test)
#         ConfusionMatrixDisplay(cm,display_labels=y.unique()).plot()
#         plt.xticks(rotation=90)

        
        mask = np.logical_or(preds_test=='SIRA',preds_test=='DERMASON')
        X_test_sira_dermason = X_test[mask]


        sira_dermason_updated_predictions = sira_dermason_model.predict(X_test_sira_dermason) 
        print("fold {} a test accuracy is {}".format(i,accuracy_score(y_test,preds_test)))
        preds_test[mask] = sira_dermason_updated_predictions
#         unique, counts = np.unique(preds_test, return_counts=True)
        # print (np.asarray((unique, counts)).T)
        
#         cm2 = confusion_matrix(y_test,preds_test)
#         ConfusionMatrixDisplay(cm2,display_labels=y.unique()).plot()
#         plt.xticks(rotation=90)
#         plt.show()               
        print("fold {} a test accuracy is {}".format(i,accuracy_score(y_test,preds_test)))

        i+=1


In [ ]:
def objective(trial):
    

    tree_method = ['exact','approx','hist']
    boosting_list = ['gblinear','gbtree']

#     feature_selector_list = ['cyclic','shuffle','random','greedy','thrifty']
#     param ={
# #           'n_estimators':trial.suggest_int('n_estimators', 30,300),
# #           'max_depth':trial.suggest_int('max_depth',1,16),
# #           'reg_lambda':trial.suggest_int('reg_lambda', 1,15),
#          "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
#         "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
#         'iterations':trial.suggest_int('iterations', 50,700),
#         'learning_rate':trial.suggest_uniform("learning_rate", 0.01, 0.3),
#         'random_strength':trial.suggest_int("random_strength", 1,10),
# #         'max_bin':trial.suggest_categorical('max_bin', [2,3,4,5,6,8,10,20,30]),
# #         'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),        
#         "colsample_bylevel": trial.suggest_uniform("colsample_bylevel", 0.1, 1),
#         'od_type' : "Iter",
#         'od_wait' : 30,
#         "depth": trial.suggest_int("max_depth", 2,7),
#         "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 10),
# #          'task_type' : 'GPU'
#         "thread_count": -1
#       }
    param = {
        "verbosity": 0,
        'n_estimators':trial.suggest_int('n_estimators', 30,300),
        'max_depth':trial.suggest_int('max_depth',4,50),

        # use exact for small dataset.
        "tree_method": trial.suggest_categorical("tree_method", ['exact','approx','hist']),
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        'learning_rate':trial.suggest_loguniform("learning_rate", 0.01, 0.3),

    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    model = xgb.XGBClassifier(random_state = 42, verbose= 0,**param)  

    i=1
    scores=[]
    for train_index, test_index in kf.split(X,y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            model.fit(X_train,y_train)
#             preds_train = model.predict(X_train)
            preds_test = model.predict(X_test)

            mask = np.logical_or(preds_test=='SIRA',preds_test=='DERMASON')
            X_test_sira_dermason = X_test[mask]
            sira_dermason_updated_predictions = sira_dermason_model.predict(X_test_sira_dermason) 
            
            preds_test[mask] = sira_dermason_updated_predictions
            scores.append(accuracy_score(y_test,preds_test))
            i+=1
            preds_test[mask] = sira_dermason_updated_predictions
            # unique, counts = np.unique(preds_test, return_counts=True)
            # print (np.asarray((unique, counts)).T)
    avg_acc  = np.mean(scores)
    print("avg test accuracy is {}".format(avg_acc))

    return avg_acc
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
from datetime import datetime

def get_current_date_time():
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")
    return dt_string

    

def generate_submission(ID,preds,model_name,acurracy):
    current_date_time = get_current_date_time()
    file_name = model_name+"_"+str(acurracy)+"_"+current_date_time+".csv"
    file_path = r'../submissions/'
    submission = pd.DataFrame({"ID":ID, "y":preds})
    print(file_path+file_name)
    submission.to_csv(os.path.join(file_path,file_name), index=False)
    print(submission.shape)


In [ ]:
# # param = {'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'iterations': 655,
# #           'learning_rate': 0.15802933328209717, 'random_strength': 10,
# #           'colsample_bylevel': 0.7201309776121098, 'max_depth': 4, 'l2_leaf_reg': 6}

# param = {'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'iterations': 560,
#  'learning_rate': 0.2999043265116359, 'random_strength': 4, 
#  'colsample_bylevel': 0.7863645192866875, 'max_depth': 2, 'l2_leaf_reg': 1}
param = {'n_estimators': 254, 'max_depth': 30, 'tree_method': 'approx',
         'booster': 'dart', 'lambda': 3.6805271258795584e-06,
         'alpha': 0.0017702781415986956, 'subsample': 0.5029439162420813,
         'colsample_bytree': 0.711692717837368, 'learning_rate': 0.28897901718740016,
         'min_child_weight': 2, 'eta': 1.2800474076575452e-05, 'gamma': 4.449002311878155e-06,
         'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 
         'rate_drop': 0.20997034391432784, 'skip_drop': 1.2586700209645195e-08}
# X,y = features_target(train,features)
# model2 = SVC()   
model2= xgb.XGBClassifier(random_state = 42, verbosity= 0, **param)  
model2.fit(X,y)


test = mm_test_df[features]
out = model2.predict(test)

unique, counts = np.unique(out, return_counts=True)
print (np.asarray((unique, counts)).T)


mask = np.logical_or(out=='SIRA',out=='DERMASON')
X_test_sira_dermason = test[mask]


sira_dermason = train[(y == 'SIRA') | (y == 'DERMASON') ]
# sira_dermason_model = ctb.CatBoostClassifier(random_state = 42, verbose=0,thread_count=-1)
# X_sira_dermason,y_sira_dermason = features_target(sira_dermason,features)
sira_dermason_model.fit(X_sira_dermason,y_sira_dermason)
sira_dermason_updated_predictions = sira_dermason_model.predict(X_test_sira_dermason)


out[mask] = sira_dermason_updated_predictions
unique, counts = np.unique(out, return_counts=True)
print (np.asarray((unique, counts)).T)

# study.best_value


In [ ]:
a = pd.read_csv('a.csv')
a = a.y
a.value_counts()

In [ ]:
test['ID'].shape

In [ ]:
model_name='mix'
acc = 0.1
generate_submission(ids,out.reshape(-1,),model_name,acc)